Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB10

Use reinforcement learning to devise a tic-tac-toe player.

### Deadlines:

* Submission: Sunday, December 17 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Dies Natalis Solis Invicti ([CET](https://en.wikipedia.org/wiki/Sol_Invictus))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [93]:
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice
from copy import deepcopy
import functools

from tqdm.auto import tqdm
import numpy as np

In [94]:
#Here i generate all possible symmetry board of tic tac toe
MAGIC = [2, 7, 6, 9, 5, 1, 4, 3, 8]
MAGIC_90=[6,1,8,7,5,3,2,9,4]
MAGIC_180=[8,3,4,1,5,9,6,7,2]
MAGIC_270=[4,9,2,3,5,7,8,1,6]
MAGIC_MIRROR=[4,3,8,9,5,1,2,7,6]
MAGIC_MIRROR_90=[2,9,4,7,5,3,6,1,8]
MAGIC_MIRROR_180=[6,7,2,1,5,9,8,3,4]
MAGIC_MIRROR_270=[8,1,6,3,5,7,4,9,2]



In [95]:
from collections import namedtuple

State = namedtuple('State', ['x', 'o'])
#i use compare function for select one of all possible configuration of one state
#i compare in this way: if comapre firstly first element of self.x with first element of other.x  if are the same i compare the next element of x
#if all element of x are the same we do samething with o
def compare(self, other):
    for elem_self, elem_other in zip(self.x, other.x):
        if elem_self < elem_other:
            return -1
        elif elem_self > elem_other:
            return 1

    for elem_self, elem_other in zip(self.o, other.o):
        if elem_self < elem_other:
            return -1
        elif elem_self > elem_other:
            return 1

    return 0
class CustomState(namedtuple('State', ['x', 'o'])):
    #for see if one configuration is equivalent to another one we extract indices and then we search those value in all confgiuration matrix
    #if one is the same the configurations are the same
    def __eq__(self, other):
        indexes=find_indices(self)
        #print("gli indici sono",indexes)
        for magic in [MAGIC,MAGIC_90,MAGIC_180,MAGIC_270,MAGIC_MIRROR,MAGIC_MIRROR_180,MAGIC_270]:
            
            val =get_values_by_keys(indexes,magic)
            #print(val, other)
            if (sorted(val.x) == sorted(other.x) and sorted(val.o) == sorted(other.o)):
                return True
        
        return False
    #for the hash we generate all possible configuration from one state and then we re order the vector with comapre fucntion and extract first element
    #in this way we have always the same configuration 
    def __hash__(self):
        vett=[]
        indexes=find_indices(self)
        for i,magic in enumerate([MAGIC,MAGIC_90,MAGIC_180,MAGIC_270,MAGIC_MIRROR,MAGIC_MIRROR_180,MAGIC_270]):
            val =get_values_by_keys(indexes,magic)
            #print(i)
            vett.append(val)
        tmp=sorted(vett, key=functools.cmp_to_key(compare))
        return hash(str(State(tmp[0].x, tuple(tmp[0].o))))  
    #we use this for convert one state in a common configuration
    def reverse(self):
            vett=[]
            indexes=find_indices(self)
            for i,magic in enumerate([MAGIC,MAGIC_90,MAGIC_180,MAGIC_270,MAGIC_MIRROR,MAGIC_MIRROR_180,MAGIC_270]):
                val =get_values_by_keys(indexes,magic)
                #print(i)
                vett.append(val)
            tmp=sorted(vett, key=functools.cmp_to_key(compare))
            return tmp[0]
   
       

# Esempio di utilizszo:
    

CustomState(set([1,3]),set([5])).__hash__()

-9204416112096060735

In [96]:
#we pass the MAGIC vector and indexes and it give us the value 
def get_values_by_keys(key_list:CustomState, vector):
    result_values = CustomState(set(),set())
    for key in key_list.x:
            try:
                value = vector[key]
                result_values.x.add(value)
            except IndexError:
                pass
    
    for key in key_list.o:
            try:
                value = vector[key]
                result_values.o.add(value)
            except IndexError:
                pass   
    return result_values



In [97]:
#give us the indices of the value
def find_indices(pos:State):
    state_tmp=CustomState(set([]),set([]))
    for i,val in enumerate(pos.x):
        #print(val)
        try:
            index = MAGIC.index(val)
            state_tmp.x.add(index)
        except ValueError:
            #print(val)

            print("errore")
            pass
    for i,val in enumerate(pos.o):
        #print(val)
        try:
            index = MAGIC.index(val)
            state_tmp.o.add(index)
        except ValueError:
            print(val)

            print("errore")
            pass
    
    return state_tmp

In [98]:
def print_board(pos):
    """Nicely prints the board"""
    board=""
    for r in range(3):
        board+='\n'

        for c in range(3):
            i = r * 3 + c
            if MAGIC[i] in pos.x:
                print('X', end='')
                board+='X'
            elif MAGIC[i] in pos.o:
                print('O', end='')
                board+='O'
            else:
                print(MAGIC[i], end='')
                board+=str(MAGIC[i])
        print()
    print()
    return board

In [99]:
def win(elements):
    """Checks is elements is winning"""
    return any(sum(c) == 15 for c in combinations(elements, 3))
def block_win_adv(state_adv,pick):
     for c in combinations(state_adv, 2):
         if 15-sum(c) ==pick:
             return 1
     return 0    
def trap_condition(user_state,adversarial_state):
        cnt=0
        for c in combinations(user_state,2):
             val =15-sum(c)

             if val not in adversarial_state and val>0:
                  cnt+=1
                  if cnt>=2:
                       return 0.5         
        return 0    
def state_value(pos: State):
    """Evaluate state: +1 first player wins"""
    if win(pos.x):
        return 1
    elif win(pos.o):
        return -1
    else:
    
        return 0

    

In [100]:
def random_move(available):

    x = choice(list(available))

        
    return x

### Q-Learning
The reward are:
- 1 for win
- 0.75 if we block adversarial win
- 0.5 if we make a trap
and viceversa but negative for adversarial

We've implemented the symmetry for board of tic tac toe, in this way instead of save all possible configuration we save only one configuration, we chose the common configuration with reverse function (from a state we generate all possible configuration and take the first after that we re order the configuration using compare function for key). In this way we pass from a dictionary of 5k element (da verificare) to a dictionary of 1k element (da verificare)

In [101]:
value_dictionary = {}

hit_state = defaultdict(int)
epsilon = 0.001
""""
the reward are 1 for win
-1 for lose
0.75 if we block adversarial win
0.5 if we make a trap
"""

for steps in tqdm(range(5000)):
    state = CustomState(set(), set())
    cnt=0
    while (state.x.union(state.o)!=set(range(1,10)) and state_value(state)==0) :
        state_tmp=deepcopy(state)
        state=state.reverse()#here i make a common rapresentation for the state  
        state_tmp=state_tmp.reverse()
        action = random_move(set(range(1,10))-(state.x.union(state.o)))
        player=cnt%2
        cnt+=1
        #print(cnt)
        if player==1:
            state_tmp.x.add(action)
            #action=(state_tmp.reverse().x-state.reverse().x).pop()
            state_tmp=state_tmp.reverse()
            reward=state_value(state_tmp)
           
            if reward==0:
                ##if there is no a win we search if we can block adversarial win
                #if we can't reward=0, and search if we can make a trap condition
                reward=block_win_adv(state_tmp.o,action)*0.75
            if reward==0:
                reward=trap_condition(state_tmp.x,state_tmp.o)
    
            if state not in value_dictionary:
                value_dictionary[state]={action:0.}
            elif action not in value_dictionary[state]:
                value_dictionary[state][action]=0.

            if state_tmp not in value_dictionary:
                value_dictionary[state_tmp]={action:0.}
            elif action not in value_dictionary[state_tmp]:
                value_dictionary[state_tmp][action]=0.
            value_dictionary[state][action]=(1-0.1)*value_dictionary[state][action]+0.1*(reward+0.7* max(value_dictionary[state_tmp].values()))
            state=deepcopy(state_tmp)
        
        else:
            state_tmp.o.add(action)
            #action=(state_tmp.reverse().o-state.reverse().o).pop()
            state_tmp=state_tmp.reverse()

            reward=state_value(state_tmp)
            if reward==0:
                reward=-block_win_adv(state_tmp.x,action)*0.75
            if reward==0:
                reward=-trap_condition(state_tmp.o,state_tmp.x)
            if state not in value_dictionary:
                value_dictionary[state]={action:0.}
            elif action not in value_dictionary[state]:
                value_dictionary[state][action]=0.

            if state_tmp not in value_dictionary:
                value_dictionary[state_tmp]={action:0.}
            elif action not in value_dictionary[state_tmp]:
                value_dictionary[state_tmp][action]=0.
           
            value_dictionary[state][action]=(1-0.1)*value_dictionary[state][action]+0.1*(reward+0.7* min(value_dictionary[state_tmp].values()))
            
            state=deepcopy(state_tmp)
        

  0%|          | 0/5000 [00:00<?, ?it/s]

In [102]:
def stampa_dizionario(dizionario, livello=0):
    spazi = "  " * livello
    for chiave, valore in dizionario.items():
        if isinstance(valore, dict):
            print(f"{spazi}{chiave}:")
            stampa_dizionario(valore, livello + 1)
        else:
            print(f"{spazi}{chiave}: {valore}")



stampa_dizionario(value_dictionary)
print(len(value_dictionary))

CustomState(x=set(), o=set()):
  5: 0.0
  4: 0.0
  8: 0.0
  9: 0.0
  6: 0.0
  3: 0.0
  1: 0.0
  7: 0.0
  2: 0.0
CustomState(x=set(), o={5}):
  5: 0.0
  3: 0.0
  6: 0.0
  1: 0.0
  9: 0.0
  7: 0.0
  4: 0.0
  8: 0.0
  2: 0.0
CustomState(x={1}, o={5}):
  3: 0.0
  4: -0.02101342555723757
  9: 0.0
  1: 0.0
  2: -0.018742137952203215
  7: 0.0
  8: 0.0
  6: 0.0
CustomState(x={1}, o={2, 5}):
  4: 0.0
  2: 0.0
  6: 0.0
  3: 0.0
  5: 0.0
  8: 0.0
  7: -0.03357735052073888
  9: 0.0
CustomState(x={1, 2}, o={4, 5}):
  4: 0.0
  8: -0.26085155000000004
  7: -0.343094701955
  3: -0.32566077995000003
  1: 0.0
  6: -0.68618940391
  9: 0.0
CustomState(x={1, 2}, o={8, 4, 5}):
  8: 0.0
  6: 0.45943463325000006
  5: 0.0
  3: 0.3071325000000001
  4: 0.0
  9: 0.0
  7: 0.20475500000000002
CustomState(x={1, 2, 6}, o={8, 4, 5}):
  6: 0.0
  7: -0.25792500000000007
  3: -0.6513215599000001
  2: 0.0
  9: 0.0
CustomState(x={1, 2, 6}, o={8, 4, 5, 7}):
  7: 0.0
  3: 0.4884911699250001
  5: 0.0
  4: 0.0
  9: 0.0
  8: 0.

In [103]:
import time
def play_against_him():
    print_board(State(set(),set()))
    state=State(set(),set())
    while (state.x.union(state.o)!=set(range(1,10)) and state_value(state)==0) :
        list_move = sorted(value_dictionary[state], key=value_dictionary[state].get)
        for i in range(len(list_move)):
            val=list_move[i]
            if val not in state.x:
                break
        state.x.add(val)
        print_board(state)
        time.sleep(1)

        numero_inserito = input("Make your move")

        try:
            #print(board+"\n")
            numero_inserito = int(numero_inserito)

            if(numero_inserito not in (set(range(1,10))-state.x.union(state.o))):
                print("mossa non valida")
                break
            else:
                    
                state.o.add(numero_inserito)
            print_board(state)
        except ValueError:
            print("Not valid.")()
    if(state_value(state)==1):
        print("RL win")
    elif(state_value(state)==-1):
        print("You win ")
    else:
        print("pair")


play_against_him()



276
951
438



TypeError: unhashable type: 'set'

In [ ]:
def play_against_random():
    print_board(CustomState(set(),set()))
    state=CustomState(set(),set())
    print("X---->RL")
    print("O----->RANDOM")
    print(range(1,10))


    while (len(state.x.union(state.o))<9 and state_value(state)==0) :
        list_move = sorted(value_dictionary[state], key=value_dictionary[state].get)
        print(list_move)
        for i in range(len(list_move)):
            val=list_move[i]
            if val not in (state.x.union(state.o)) :
                break
        state.x.add(val)
        print(len(state.x.union(state.o)))
        if len(state.x.union(state.o))==9:
            break

        move=random_move(set(range(1,10))-(state.x.union(state.o)))
        state.o.add(move)
        state=state.reverse()
        print(state)
        print_board(state)
    
    if(state_value(state)==1):
        print("you win")
    elif (state_value(state)==-1):
        print("random win")
    else:
        print("pair")

        

play_against_random()        

        



276
951
438

X---->RL
O----->RANDOM
range(1, 10)
[CustomState(x=set(), o=set()), CustomState(x=set(), o=set()), CustomState(x=set(), o=set()), CustomState(x=set(), o=set()), CustomState(x=set(), o=set()), CustomState(x=set(), o=set()), CustomState(x=set(), o=set())]
[CustomState(x=set(), o=set()), CustomState(x=set(), o=set()), CustomState(x=set(), o=set()), CustomState(x=set(), o=set()), CustomState(x=set(), o=set()), CustomState(x=set(), o=set()), CustomState(x=set(), o=set())]
[6, 2, 9, 3, 1, 7, 5, 8, 4]
1
[CustomState(x={2}, o={4}), CustomState(x={2}, o={6}), CustomState(x={2}, o={6}), CustomState(x={4}, o={2}), CustomState(x={6}, o={8}), CustomState(x={8}, o={4}), CustomState(x={8}, o={6})]
CustomState(x={2}, o={4})
X76
951
O38

[CustomState(x={2}, o={4}), CustomState(x={4}, o={2}), CustomState(x={4}, o={8}), CustomState(x={4}, o={8}), CustomState(x={6}, o={2}), CustomState(x={6}, o={8}), CustomState(x={8}, o={6})]
[CustomState(x={2}, o={4}), CustomState(x={4}, o={2}), CustomState

In [ ]:
def is_symmetric(move1, move2):
    # Definisci la mappa di simmetria rispetto a una rotazione di 180 gradi
    symmetry_map = {'1': '9', '2': '8', '3': '7', '4': '6', '5': '5', '6': '4', '7': '3', '8': '2', '9': '1'}

    # Verifica se le mosse sono simmetriche rispetto a una rotazione di 180 gradi
    return all(symmetry_map.get(digit) == move2[i] for i, digit in enumerate(move1))

# Esempio di utilizzo:
move_user1 = '27'
move_user2 = '59'

if is_symmetric(move_user1, move_user2):
    print("Le mosse sono simmetriche rispetto a una rotazione di 180 gradi.")
else:
    print("Le mosse non sono simmetriche rispetto a una rotazione di 180 gradi.")


Le mosse non sono simmetriche rispetto a una rotazione di 180 gradi.


In [ ]:
fruits = ['apple', 'banana', 'orange']

for index, fruit in enumerate(fruits, start=1):
    print(f"Index: {index}, Fruit: {fruit}")


Index: 1, Fruit: apple
Index: 2, Fruit: banana
Index: 3, Fruit: orange
